<a href="https://colab.research.google.com/github/szambetti/INVEST_IO/blob/master/market_screener_reccom_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Questo notebook importa le raccomandazioni degli analisti da marketscreener, di default il link è di intesa san paolo ma può essere cambiato come riportato sotto

In [11]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import json
import pprint
import numpy as np

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 256 kB in 2s (151 kB/s)
Reading package lists... Done
Reading package lists... Done
B

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: use options instead of chrome_options


In [12]:
def get_tds():
  wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  #loop sulle raccomandazioni dalla pagina principale
  for i in wd.find_elements_by_css_selector('table#ALNI4.tabBody >* tr'):
    date = i.find_elements_by_css_selector('.pleft5.pright5.large30')[0].text
    reccom = i.find_elements_by_css_selector('.newsColCT.ptop3')[0].text
    link = i.find_elements_by_css_selector('.newsColCT.ptop3 > a')[0].get_attribute('href')
    output.append({'date': date, 'reccom' : reccom , 'link' : link})

In [13]:
def click_on_button(x):
  element = wd.find_element_by_css_selector(x)
  wd.execute_script("arguments[0].click();", element)

In [ ]:
#MAIN
url = 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news-broker-research/' #< sostituisci qui il link
wd.set_window_size(1920,1080)
wd.maximize_window()
wd.get(url)

output = []
next_button = 1

while next_button > 0:
  get_tds()
  try:
    button_container = 'a.nPageEndTab'
    wd.find_element_by_css_selector(button_container)
    click_on_button(button_container)
  except:
    next_button = 0

In [15]:
full_output = []
import time
def loop_single_reccomm(k):
  url = k['link']
  date = k['date']
  title = k['reccom']
  wd.get(url)
  time.sleep(abs(np.random.normal(2,1)))
  time_published = wd.find_element_by_css_selector('meta[itemprop="datePublished"]').get_attribute('content')
  desc = wd.find_element_by_id('grantexto').text
  full_output.append({'date_short' : date, 'title' : title, 'url': url, 'date_long' : time_published, 'desc' : desc})
  return full_output

for i in output:
  loop_single_reccomm(i)
print (full_output)

[{'date_short': '08/07', 'title': 'INTESA SANPAOLO S.P.A. : Upgraded to Buy by UBS', 'url': 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news/INTESA-SANPAOLO-S-P-A-Upgraded-to-Buy-by-UBS-31076623/', 'date_long': '2020-08-07T02:29:03-04:00', 'desc': "UBS's analyst Ignacio Cerezo upgrades his rating from Neutral to Buy. The target price has been lifted and is now set at EUR 2.40 compared to EUR 1.95 before."}, {'date_short': '08/05', 'title': 'INTESA SANPAOLO S.P.A. : Kepler Cheuvreux reaffirms its Buy rating', 'url': 'https://www.marketscreener.com/INTESA-SANPAOLO-SPA-68944/news/INTESA-SANPAOLO-S-P-A-Kepler-Cheuvreux-reaffirms-its-Buy-rating-31061006/', 'date_long': '2020-08-05T08:10:03-04:00', 'desc': 'Anna Maria Benassi from Kepler Cheuvreux retains his positive opinion on the stock with a Buy rating. The target price is reviewed upwards from EUR 2.10 to EUR 2.22.'}, {'date_short': '08/04', 'title': 'INTESA SANPAOLO S.P.A. : Jefferies reiterates its Buy rating', 'url': 'h

In [19]:
from pandas import DataFrame
df = DataFrame (full_output)
df.to_csv('sr_market_screener_reccom.csv')